In [1]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import when

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("dbcreation_alkemy")\
.config ("spark.sql.shuffle.partitions", "50")\
.config("spark.driver.maxResultSize","5g")\
.config ("spark.sql.execution.arrow.enabled", "true")\
.getOrCreate()

22/12/08 11:41:39 WARN Utils: Your hostname, MacBook-Pro-di-Fabiana.local resolves to a loopback address: 127.0.0.1; using 192.168.1.52 instead (on interface en0)
22/12/08 11:41:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/08 11:41:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
regular = spark.read.csv('clicks_regular_all.csv', header=True, inferSchema=True)
bidding = spark.read.csv('clicks_bidding_all.csv', header=True, inferSchema=True)
concatenato = regular.union(bidding)

22/12/08 11:41:41 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/12/08 11:41:41 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/12/08 11:41:41 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/12/08 11:41:45 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/12/08 11:41:45 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the f

22/12/08 11:41:48 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/12/08 11:41:48 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/12/08 11:41:48 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/12/08 11:41:48 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
22/12/08 11:41:48 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the f

In [4]:
concatenato = concatenato.withColumn("type", when(concatenato.type == "Regular",0) \
      .otherwise(1))

concatenato = concatenato.withColumn("position", when(concatenato.type == 0,concatenato.position + 3) \
      .otherwise(concatenato.position))

concatenato3 = concatenato.filter(
    ((concatenato.position != 0)
     & (concatenato.type != 0))
)

In [5]:
concatenato.show()

+-------------------+------+--------+---------+---------+-----+----+----------+
|               date|seller|position|price_max|price_min|price|type|product_id|
+-------------------+------+--------+---------+---------+-----+----+----------+
|2020-01-01 00:28:01|    48|    null|   5290.0|   5290.0| null|   0|    148875|
|2020-01-01 00:29:40|    48|    null|  11787.0|   9990.0| null|   0|    104538|
|2020-01-01 00:30:39|    23|    null|  16217.0|  16217.0| null|   0|    108283|
|2020-01-01 00:42:23|    48|    null|   7399.0|   6398.0| null|   0|    144718|
|2020-01-01 00:42:51|   407|    null|   3343.0|   2299.0| null|   0|    144776|
|2020-01-01 00:42:57|    24|    null|  12999.0|   9491.0| null|   0|    132634|
|2020-01-01 00:44:21|    24|    null|  10890.0|   9290.0| null|   0|    110429|
|2020-01-01 00:46:46|    24|    null|  31690.0|  28999.0| null|   0|    141943|
|2020-01-01 00:56:15|    24|    null|  43199.0|  43199.0| null|   0|    128665|
|2020-01-01 00:57:23|    24|    null|  5